## text generation with LSTMswith keras

In [1]:
def read_file(filepath):
    with open(filepath) as f:
        str_text=f.read()
        
    return str_text   

In [2]:
# tokenize and clean the text using spacey
import spacy
nlp=spacy.load('en_core_web_sm',disable=['parser', 'tagger', 'ner']) # load en i.e., english ; alse disbling some word which do not need recogination like parsing,tagger etc
#spacy.load('')


In [3]:
nlp.max_length= 1198623

In [4]:
''' defining a function and it takes some document text as a string 
and then it's going grab the text tokens. if they are not some type of punctuation
or if they are not any line. '''
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_{|}~\t\n ']


In [5]:
d= read_file('E:/study material/Deep learning/NLP/moby_dick_four_chapters.txt')

In [6]:
tokens= separate_punc(d) #calling the function

In [7]:
len(tokens)

11394

In [8]:
# we are passinh here 25 word and let the NN predict 26 word
train_len=25+1
text_sequences=[]

for i in range(train_len, len(tokens)):
    seq=tokens[i-train_len:i] # it contains sequence of 26 value in it 
                              #so that for given 25 value it can predics 26th
    
    text_sequences.append(seq)

In [9]:
print(type(text_sequences))

<class 'list'>


In [10]:
print(text_sequences[0]) # it will print 0 to 25

['call', 'me', 'ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on']


In [11]:
 print(text_sequences[1]) # it will print 1 to 26

['me', 'ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on', 'shore']


In [12]:
# to see entire in 1 sequence together
print(' '.join(text_sequences[0]))

call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on


In [13]:
print(' '.join(text_sequences[1]))

me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore


In [14]:
print(' '.join(text_sequences[2]))

ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i


In [15]:
# koros use to tokenization into numerical system
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.
C:\Users\Abhishek\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Abhishek\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Abhishek\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Abhishek\Anaconda3\lib\site-packages\tensorflow\python\framewo

In [16]:
tokenizer = Tokenizer() # let it has it's value
tokenizer.fit_on_texts(text_sequences) #  coverting text into numbers

In [17]:
sequences= tokenizer.texts_to_sequences(text_sequences)

In [18]:
print(sequences[0]) # we can say that each of the numver is ID of particular word

[964, 14, 265, 51, 263, 416, 87, 222, 129, 111, 962, 262, 50, 43, 37, 321, 7, 23, 555, 3, 150, 261, 6, 2704, 14, 24]


In [19]:
print(sequences[1]) # we can say that each of the numver is ID of particular word

[14, 265, 51, 263, 416, 87, 222, 129, 111, 962, 262, 50, 43, 37, 321, 7, 23, 555, 3, 150, 261, 6, 2704, 14, 24, 965]


In [20]:
# to see relationship between ID and word
#tokenizer.index_word--> this will give result for all words

# for one sequences
for i in sequences[0]:
    print(f"{i}:{tokenizer.index_word[i]}") # for same word will have same numberical ID 

964:call
14:me
265:ishmael
51:some
263:years
416:ago
87:never
222:mind
129:how
111:long
962:precisely
262:having
50:little
43:or
37:no
321:money
7:in
23:my
555:purse
3:and
150:nothing
261:particular
6:to
2704:interest
14:me
24:on


In [21]:
#tokenizer.word_counts--> given no of time a word came

In [22]:
# to get size of vocabulary
vocabulary_size=len(tokenizer.word_counts)
print(vocabulary_size)

2709


In [23]:
type(sequences)

list

In [24]:
#to convert sequences into numpy
import numpy as np
sequences=np.array(sequences)
print(sequences) 

[[ 964   14  265 ... 2704   14   24]
 [  14  265   51 ...   14   24  965]
 [ 265   51  263 ...   24  965    5]
 ...
 [ 960   12  168 ...  264   53    2]
 [  12  168 2703 ...   53    2 2709]
 [ 168 2703    3 ...    2 2709   26]]


In [25]:
'''we are going to do train-test split on sequence.
assuming last column as target and rest of the column as features'''
from keras.utils import to_categorical
X=sequences[:,:-1]
y=sequences[:,-1]
y=to_categorical(y,num_classes=vocabulary_size+1)
seq_len=X.shape[1] # storing number of features i.e., 25
#seq_len

In [26]:
# Now making tha model
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [27]:
def create_model(vocabulary_size,seq_len):
    model=Sequential()
    model.add(Embedding(vocabulary_size,seq_len,input_length=seq_len)) # taking input dim=vocabulary_size and output dimension=seq_len
    model.add(LSTM(50,return_sequences=True)) # randomy choosing number of neuron inside LSTM equal to 50. we can also take 150 value
    model.add(LSTM(50))
    model.add(Dense(50,activation='relu'))
    
    model.add(Dense(vocabulary_size,activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    
    model.summary()
    
    return model
                                                                             

In [28]:
model=create_model(vocabulary_size+1,seq_len) # we added 1 in vocabulary_size for cosidering 0

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 25)            67750     
_________________________________________________________________
lstm_1 (LSTM)                (None, 25, 50)            15200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 2710)              138210    
Total params: 243,910
Trainable params: 243,910
Non-trainable params: 0
_________________________________________________________________


In [29]:
from pickle import dump,load
model.fit(X,y,batch_size=512,epochs=500,verbose=1)

#odel.fit(X,y,batch_size=512,epochs=2,verbose=1) #batch size is how  many sequences we want to passin so that NN can handle and that chosen arbitarly
                                                 #  verbose give information about accuracy
# our accuracy will be very low as we have taken value of epocha only 2.    

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
11368/11368 [==============================] - ETA: 25s - loss: 7.9047 - accuracy: 0.0000e+0 - ETA: 13s - loss: 7.9041 - accuracy: 0.0068    - ETA: 9s - loss: 7.9035 - accuracy: 0.013 - ETA: 7s - loss: 7.9030 - accuracy: 0.01 - ETA: 6s - loss: 7.9023 - accuracy: 0.02 - ETA: 5s - loss: 7.9016 - accuracy: 0.02 - ETA: 4s - loss: 7.9007 - accuracy: 0.02 - ETA: 4s - loss: 7.9000 - accuracy: 0.02 - ETA: 3s - loss: 7.8989 - accuracy: 0.02 - ETA: 3s - loss: 7.8979 - accuracy: 0.02 - ETA: 2s - loss: 7.8964 - accuracy: 0.02 - ETA: 2s - loss: 7.8948 - accuracy: 0.02 - ETA: 2s - loss: 7.8925 - accuracy: 0.02 - ETA: 1s - loss: 7.8898 - accuracy: 0.02 - ETA: 1s - loss: 7.8867 - accuracy: 0.02 - ETA: 1s - loss: 7.8825 - accuracy: 0.02 - ETA: 1s - loss: 7.8765 - accuracy: 0.02 - ETA: 0s - loss: 7.8683 - accuracy: 0.02 - ETA: 0s - loss: 7.8593 - accuracy: 0.02 - ETA: 0s - loss: 7.8479 - accuracy: 0.02 - ETA: 0s - loss: 7.8342 - accuracy: 0.02 

In [30]:
# to save this model
model.save('E:/study material/Deep learning/NLP/LSTM.h5')

In [31]:
#to save tokenizer
dump(tokenizer,open('E:/study material/Deep learning/NLP/simpletokeniser','wb'))

## To generate new text using seed text

In [32]:
from keras.preprocessing.sequence import pad_sequences

In [33]:
# seed text means some text to just start off
def generate_text(model,tokenizer, seq_len, seed_text, num_gen_words): 
    #num_gen_word is number of word we want to generate after the seed_text
    # tokenzer has idea about what ID number goes to which word
    output_text=[]
    
    input_text=seed_text
    for i in range(num_gen_words):
        encoded_text=tokenizer.texts_to_sequences([input_text])[0] # [0] is use to grab first item here
        # transformed the the text data into sequences of number
        
        pad_encoded=pad_sequences([encoded_text],maxlen=seq_len,truncating='pre') 
        '''
        ->if seed_text is too long or too short then we can pad it up to 25. 
        ->for bext result seed_text lenghth should be equal to seq_len.
        ->for truncating we can either go begining 
        or after the string here it is at begning i.e, "pre"  '''

        # pred_word_ind--> predicted word index
        pred_word_ind= model.predict_classes(pad_encoded, verbose=0) [0]
        # this predicted class probability for each word
        
        predicted_word=tokenizer.index_word[pred_word_ind]
        
        input_text+=' '+predicted_word
        
        output_text.append(predicted_word)
        
        
    return ' '.join(output_text)   
    
    

In [34]:
# now to indiatate seed text, and we can do it either directly indiating 
# or choosing randomly from text_sequences we found earlier.
print(text_sequences[0])

['call', 'me', 'ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on']


In [35]:
# to chood seed text randomly-
import random
random.seed(101)
random_pick= random.randint(0,len(text_sequences))
random_seed_text=text_sequences[random_pick]
print(random_seed_text)

['and', 'throwing', 'the', 'clothes', 'to', 'one', 'side', 'he', 'really', 'did', 'this', 'in', 'not', 'only', 'a', 'civil', 'but', 'a', 'really', 'kind', 'and', 'charitable', 'way', 'i', 'stood', 'looking']


In [36]:
seed_text=' '.join(random_seed_text) # here seed_text lenghth is 25 so it will give better result
print(seed_text)

and throwing the clothes to one side he really did this in not only a civil but a really kind and charitable way i stood looking


In [37]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text, num_gen_words=10) # this will give result as we only did training on very less epochs

"in an boat and tropical 's it was you has"

In [38]:
print(' '.join(text_sequences[0]))

call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on


In [39]:
print(' '.join(text_sequences[15]))

money in my purse and nothing particular to interest me on shore i thought i would sail about a little and see the watery part of


In [40]:
seed_text=' '.join(text_sequences[0]) #second example

In [41]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text, num_gen_words=10) # this will give result as we only did training on very less epochs

'shore i thought i would sail about a little and'

In [42]:
seed_text=' '.join(text_sequences[3]) #second example

In [43]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text, num_gen_words=10) # this will give result as we only did training on very less epochs

'i would sail about a little and somewhat now no'

In [44]:
print(seed_text)

some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i thought
